# Modelación del problema de ciudades Instancias


In [2]:
import re
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [27]:
# Path al archivo txt
#file_path = 'Inst29.txt'
#file_path = 'xql662.tsp'
#file_path = 'djb2036.tsp'
file_path = 'fnc19402.tsp'

# Lista para almacenar las coordenadas de las ciudades
cities = []

# Abrir el archivo en modo lectura
with open(file_path, 'r') as file:
    lines = file.readlines()

    # Iterar sobre cada línea del archivo
    for line in lines:
        # Buscar líneas que contengan dos números decimales
        match = re.match(r'\s*(\d+)\s+((\d+\.\d+)|(\d+))\s+((\d+\.\d+)|(\d+))\s*', line)
        if match:
            city_id = int(match.group(1))
            x_coord = float(match.group(2)) if match.group(3) else int(match.group(2))
            y_coord = float(match.group(5)) if match.group(6) else int(match.group(5))
            cities.append((city_id, x_coord, y_coord))

# Crear DataFrame de pandas
data = pd.DataFrame(cities, columns=['City', 'x', 'y'])

In [28]:
data = data.iloc[1:]

data = data.reset_index()

data

,index,City,x,y
0,1,2,0,283
1,2,3,0,389
2,3,4,0,489
3,4,5,0,817
4,5,6,2,102
...,...,...,...,...
19396,19397,19398,539,974
19397,19398,19399,539,975
19398,19399,19400,539,976
19399,19400,19401,539,977


In [29]:
# Código para mostrar el camino de las ciudades.
def animate_solution(solution_df, title, filename):
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'ro-')  # Path line
    ax.scatter(solution_df['x'], solution_df['y'], color='blue')  # City points
    for _, row in solution_df.iterrows():
        ax.text(row['x'], row['y'], row['City'])  # City labels

    ax.set_xlim(min(solution_df['x']) - 1, max(solution_df['x']) + 1)
    ax.set_ylim(min(solution_df['y']) - 1, max(solution_df['y']) + 1)
    ax.set_title(title)
    ax.grid(True)

    def init():
        line.set_data([], [])
        return line,

    def animate(i):
        x_values = solution_df['x'].iloc[:i + 1]
        y_values = solution_df['y'].iloc[:i + 1]
        line.set_data(x_values, y_values)
        return line,

    anim = FuncAnimation(fig, animate, init_func=init,
                         frames=len(solution_df), interval=1000, blit=True)

    anim.save(filename, writer='pillow', fps=2)

In [30]:
# Sample DataFrame, replace this with your actual data
df = pd.DataFrame(data)
df

,index,City,x,y
0,1,2,0,283
1,2,3,0,389
2,3,4,0,489
3,4,5,0,817
4,5,6,2,102
...,...,...,...,...
19396,19397,19398,539,974
19397,19398,19399,539,975
19398,19399,19400,539,976
19399,19400,19401,539,977


# Heurística de Búsqueda Local

In [26]:
def calculate_distance(city1, city2):
    return np.sqrt((city1['x'] - city2['x'])**2 + (city1['y'] - city2['y'])**2)

def local_search(data):
    #Para cantidad de ciudades grandes, se empieza en una solución random.
    current_solution = data.sample(frac=1).reset_index(drop=True)
    best_solution = current_solution.copy()
    current_distance = sum([calculate_distance(current_solution.iloc[i], current_solution.iloc[i+1]) for i in range(len(current_solution)-1)])
    best_distance = current_distance

    improved = True
    while improved:
        improved = False
        # Empezar a recorrer el inicio del subarreglo
        for i in range(1, len(current_solution)-2):
            #print(i)
            # Empezar a recorrer el fin del subarreglo
            for j in range(i+1, len(current_solution)-1):
                new_solution = current_solution.copy()
                # Invertir valores del subarreglo (del i al j incluyentes)
                print(new_solution)
                new_solution.iloc[i:j+1] = new_solution.iloc[i:j+1].values[::-1]
                print(new_solution)
                #new_distance = sum([calculate_distance(new_solution.iloc[i], new_solution.iloc[i+1]) for i in range(len(new_solution)-1)])
                # Calcular la nueva distancia es lo mismo lo de arriba que lo de abajo.
                # La de abajo calcula la distancia de las orillas, que es la distancia que cambia. 
                new_distance = current_distance + calculate_distance(new_solution.iloc[i - 1], new_solution.iloc[i])
                new_distance += calculate_distance(new_solution.iloc[j], new_solution.iloc[j+1])
                new_distance -= calculate_distance(current_solution.iloc[i - 1], current_solution.iloc[i])
                new_distance -= calculate_distance(current_solution.iloc[j], current_solution.iloc[j+1])
                # Checat si la la distancia de la solución invertida es menor a la distancia de la solución no invertida
                if new_distance < current_distance:
                    current_solution = new_solution
                    current_distance = new_distance
                    improved = True
                    # Si la distancia es menor a best_d, se cambia. 
                    if new_distance < best_distance:
                        best_solution = current_solution.copy()
                        best_distance = new_distance

        # Completar el ciclo cerrado
        best_distance += calculate_distance(best_solution.iloc[-1], best_solution.iloc[0])
        best_solution.loc[len(best_solution)] = best_solution.iloc[0]

    return best_solution, best_distance

# Uso de la heurística de búsqueda local
start_time = time.time()
best_solutionBL, best_distanceBL = local_search(data)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Tiempo transcurrido: {elapsed_time} segundos")
print(best_distanceBL)

    index  City       x       y
0      24    25  1280.0   790.0
1      21    22   490.0   500.0
2      17    18   460.0   860.0
3      10    11   840.0   550.0
4      23    24  1260.0  1500.0
5      25    26   490.0  2130.0
6      14    15   750.0   900.0
7      22    23  1840.0  1240.0
8       6     7  1650.0   650.0
9      12    13   970.0  1340.0
10      1     2   630.0  1660.0
11      9    10   710.0  1310.0
12      4     5   750.0  2030.0
13     11    12  1170.0  2300.0
14     13    14   510.0   700.0
15     18    19  1040.0   950.0
16     26    27  1460.0  1420.0
17     28    29   360.0  1980.0
18      5     6  1030.0  2070.0
19     16    17   230.0   590.0
20      7     8  1490.0  1630.0
21      8     9   790.0  2260.0
22     20    21   830.0  1770.0
23      2     3    40.0  2090.0
24     15    16  1280.0  1200.0
25     19    20   590.0  1390.0
26     27    28  1260.0  1910.0
27      3     4   750.0  1100.0
    index  City       x       y
0      24    25  1280.0   790.0
1      1

In [ ]:
# Ejemplo de uso
animate_solution(best_solutionBL, 'Búsqueda Local camino más corto (Total Distance: {:.2f})'.format(best_distanceBL), 'Búsqueda_Local_Solution.gif')

In [ ]:
best_solutionBL

# Búsqueda Tabú

In [74]:
def calculate_distance(city1, city2):
    return np.sqrt((city1['x'] - city2['x'])**2 + (city1['y'] - city2['y'])**2)

def tabu_search(data, tabu_list_size, max_iterations):
    current_solution = data.sample(frac=1).reset_index(drop=True)
    best_solution = current_solution.copy()
    current_distance = sum([calculate_distance(current_solution.iloc[i], current_solution.iloc[i+1]) for i in range(len(current_solution)-1)])
    best_distance = current_distance
    tabu_list = []

    for k in range(max_iterations):
        print(k)
        best_candidate = None
        best_candidate_distance = float('inf')
        mini_lista_tabu = []
        for i in range(1, len(current_solution)-2):
            for j in range(i+1, len(current_solution)-1):
                mini_lista = current_solution.iloc[i+1:j+1].values.tolist()
                # mini_list es el posible movimiento a hacer, verificar no este en tabu_list
                if mini_lista not in tabu_list:
                    new_solution = current_solution.copy()
                    new_solution.iloc[i:j+1] = new_solution.iloc[i:j+1].values[::-1]
                    #new_distance = sum([calculate_distance(new_solution.iloc[i], new_solution.iloc[i+1]) for i in range(len(new_solution)-1)])
                    # Calcular la nueva distancia es lo mismo lo de arriba que lo de abajo.
                    # La de abajo calcula la distancia de las orillas, que es la distancia que cambia. 
                    new_distance = current_distance + calculate_distance(new_solution.iloc[i - 1], new_solution.iloc[i])
                    new_distance += calculate_distance(new_solution.iloc[j], new_solution.iloc[j+1])
                    new_distance -= calculate_distance(current_solution.iloc[i - 1], current_solution.iloc[i])
                    new_distance -= calculate_distance(current_solution.iloc[j], current_solution.iloc[j+1])
                    if new_distance < best_candidate_distance:
                        best_candidate = new_solution.copy()
                        best_candidate_distance = new_distance
                    else:
                        tabu_list.append(mini_lista)
                        
        if best_candidate is None:
            break
        # Mejor solución de la iteración
        current_solution = best_candidate
        current_distance = best_candidate_distance
        #Checar si la solución de la iteración es menor a la mejor solución
        if current_distance < best_distance:
            best_solution = current_solution.copy()
            best_distance = current_distance
            
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)

    # Completar el ciclo cerrado
    best_distance += calculate_distance(best_solution.iloc[-1], best_solution.iloc[0])
    best_solution.loc[len(best_solution)] = best_solution.iloc[0]

    return best_solution, best_distance

# Uso de la búsqueda tabú
tabu_list_size = 16
max_iterations = 150
start_time = time.time()
best_solutionTabu, best_distanceTabu = tabu_search(data, tabu_list_size, max_iterations)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Tiempo transcurrido: {elapsed_time} segundos")
print(best_distanceTabu)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108


KeyboardInterrupt: 

In [ ]:
animate_solution(best_solutionTabu, 'Búsqueda Tabu camino más corto (Total Distance: {:.2f})'.format(best_distanceTabu), 'Tabu_Solution.gif')

# Colonia de Hormigas

In [18]:
def calculate_distance(city1, city2):
    return np.sqrt((city1['x'] - city2['x'])**2 + (city1['y'] - city2['y'])**2)

def create_distance_matrix(cities):
    num_cities = len(cities)
    distance_matrix = np.zeros((num_cities, num_cities))
    k = 1
    sum = 0
    max_distance = 0
    for i in range(num_cities):
        for j in range(i+1, num_cities):
            distance_matrix[i][j] = calculate_distance(cities[i], cities[j])
            distance_matrix[j][i] = distance_matrix[i][j]
            k = k +1
            sum = sum + distance_matrix[i][j]
            if distance_matrix[i][j] > max_distance:
                max_distance = distance_matrix[i][j]
    average = sum / k
    return distance_matrix, average, max_distance


In [19]:
cities = data.to_dict('records')
distance_matrix, average, max_distance = create_distance_matrix(cities)

In [ ]:
def ant_colony_optimization(cities,distance_matrix, average, max_distance, num_ants, num_iterations, alpha=1.7, beta=10, evaporation_rate=0.8):
    num_cities = len(cities)
    #distance_matrix, average, max_distance = create_distance_matrix(cities)
    #print(distance_matrix)
    #pheromone_matrix = np.ones((num_cities, num_cities))
    #print(max_distance)
    # Modificar la matriz de feromonas para que los valores vayan de 1 a .7
    #print(1 - (max_distance/ ( 1.5 * max_distance))** 3) = 1 - (2/3)** 3 = 1 - .3 = .7
    pheromone_matrix = 1 - (distance_matrix / ( 1.5 * max_distance))** 3
    #print(pheromone_matrix)
    best_path = None
    best_distance = float('inf')

    for i in range(num_iterations):
        print(i)
        # flag y counter se utilizaran para modificar parámetros en caso de que no haya una solución encontrada después de varias iteraciones
        flag = False
        counter = 0
        for ant in range(num_ants):
            visited = [False] * num_cities
            current_city = np.random.randint(num_cities)
            visited[current_city] = True
            path = [current_city]
            total_distance = 0

            for _ in range(num_cities - 1):
                next_city = None
                probabilities = np.zeros(num_cities)

                for city in range(num_cities):
                    if not visited[city]:
                        probabilities[city] = (pheromone_matrix[current_city][city] ** alpha) * ((1 / distance_matrix[current_city][city]) ** beta)

                probabilities = probabilities / np.sum(probabilities)
                next_city = np.random.choice(range(num_cities), p=probabilities)

                path.append(next_city)
                visited[next_city] = True
                total_distance += distance_matrix[current_city][next_city]
                current_city = next_city

            
            total_distance += distance_matrix[path[-1]][path[0]]
            path.append(path[0])

            if total_distance < best_distance:
                best_distance = total_distance
                best_path = path
                print(best_distance)
                flag = True

        if flag == True:
            i += 1
        if i >= 20:
            evaporation_rate = np.random.uniform(.4, .7)
        if i >= 30:
            a = evaporation_rate = np.random.uniform(-.3, .3)
            beta *= (1 + a)
            
            
        pheromone_matrix *= (1 - evaporation_rate)

        for i in range(num_cities):
            #print("Distance")
            #print(distance_matrix[best_path[i]][best_path[(i + 1) % num_cities]])
            #print("Feromone Before")
            #print(pheromone_matrix[best_path[i]][best_path[(i + 1) % num_cities]])
            x = distance_matrix[best_path[i]][best_path[(i + 1) % num_cities]]
            a = np.exp(-x) 
            pheromone_matrix[best_path[i]][best_path[(i + 1) % num_cities]] += ((50 / best_distance) + a )
            #print("Feromone After")
            #print(pheromone_matrix[best_path[i]][best_path[(i + 1) % num_cities]])

    return best_path, best_distance

# Ejemplo de uso
cities = data.to_dict('records')
#Matriz de distancia obtenida en función arriba
distance_matrix, average, max_distance 
num_ants = 10
num_iterations = 50
start_time = time.time()
best_path, best_distance = ant_colony_optimization(cities, distance_matrix, average, max_distance, num_ants, num_iterations)
end_time = time.time()
elapsed_time = end_time - start_time

0
80525.19723656875
79497.41164742546


##### print(f"Tiempo transcurrido: {elapsed_time} segundos")
print("Mejor camino encontrado:", best_path)
print("Distancia total:", best_distance)

In [42]:
distance_matrix, average, max_distance = create_distance_matrix(cities)
total_distance = 0
for i in range (len(best_path)-1):
    print(i)
    total_distance += distance_matrix[best_path[i]][best_path[i+1]]
    
total_distance


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


34492.34183399084

In [185]:
def animate_solution(solution_df, path, title, filename):
    fig, ax = plt.subplots()
    line, = ax.plot([], [], 'ro-')  # Path line
    ax.scatter(solution_df['x'], solution_df['y'], color='blue')  # City points
    for _, row in solution_df.iterrows():
        ax.text(row['x'], row['y'], row['City'])  # City labels

    #ax.set_xlim(min(solution_df['x']) - 1, max(solution_df['x']) + 1)
    #ax.set_ylim(min(solution_df['y']) - 1, max(solution_df['y']) + 1)
    ax.set_xlim(0, 500)  # Establecer límites x de 0 a 700
    ax.set_ylim(0, 500)
    ax.set_title(title)
    ax.grid(True)

    def init():
        line.set_data([], [])
        return line,

    def animate(i):
        x_values = [solution_df['x'][solution_df['City'] == path[j]] for j in range(i + 1)]
        y_values = [solution_df['y'][solution_df['City'] == path[j]] for j in range(i + 1)]
        line.set_data(x_values, y_values)
        return line,

    anim = FuncAnimation(fig, animate, init_func=init,
                         frames=len(path), interval=1000, blit=True)

    anim.save(filename, writer='pillow', fps=2)


In [149]:
animate_solution(df, best_path, 'Búsqueda Colonia de Hormigas camino más corto (Total Distance: {:.2f})'.format(best_distance), 'Tabu_Solution.gif')

C:\Users\Diana\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\cbook.py:1699: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return math.isfinite(val)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (498,) + inhomogeneous part.

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (498,) + inhomogeneous part.

<Figure size 640x480 with 1 Axes>